## Present proof

In [1]:
import os
import time
import datetime
import base64
from pprint import pprint
from dotenv import load_dotenv
from termcolor import colored,cprint

from prism_agent_open_api_specification_client import Client
from prism_agent_open_api_specification_client.types import Response
from prism_agent_open_api_specification_client.models import ConnectionCollection,Connection,ConnectionInvitation,CreateConnectionRequest,AcceptConnectionInvitationRequest
from prism_agent_open_api_specification_client.api.connections_management import get_connections,get_connection,create_connection,accept_connection_invitation

from prism_agent_open_api_specification_client.models import PresentationStatus, ErrorResponse, Proof, ProofRequestAux, PublicKeyJwk, RequestPresentationInput, RequestPresentationOutput, RequestPresentationAction, RequestPresentationActionAction
from prism_agent_open_api_specification_client.api.present_proof import get_presentation, get_all_presentation, request_presentation, update_presentation

### Ultilitary functions

In [2]:
def get_invitation_str(connection):
    parts = connection.invitation.invitation_url.split("=")
    return parts[1]

def find_proof_request_by_state(client, state):
    proof_requests: Response[PresentationStatus] = get_all_presentation.sync(client=client)
    
    for proof_request in proof_requests:
        if(proof_request.status == state):
            return proof_request
    return None 

def print_proof_request(proof_request):
    print(f"presentation_id: {proof_request.presentation_id}")
    if hasattr(proof_request, "status"):
        print(f"status:          {proof_request.status}")
    if hasattr(proof_request, "connection_id"):
        print(f"connection_id:   {proof_request.connection_id}")

def print_connection(connection):
    print(f"connection_id: {connection.connection_id}")
    print(f"state:         {connection.state}")
    print(f"label:         {connection.label}")
    print(f"my_did:        {connection.my_did}")
    print(f"their_did:     {connection.their_did}")
    print(f"created_at:    {connection.created_at}")

### Client instances

We will create two separate clients, one for the verifier and one for the holder, in order to establish a connection between the two.

note: remember to update the file variables.env with the URLs and API keys provided to you.


In [3]:
load_dotenv("../BetaProgram/variables.env")
verifierApiKey = os.getenv('VERIFIER_APIKEY')
verifierUrl = os.getenv('VERIFIER_URL')

holderApiKey = os.getenv('HOLDER_APIKEY')
holderUrl = os.getenv('HOLDER_URL')

verifier_client = Client(base_url=verifierUrl, headers={"apiKey": verifierApiKey})
holder_client = Client(base_url=holderUrl, headers={"apiKey": holderApiKey})

### Create connection

For details on this see "Example 01 - Connections"

In [4]:
print("Please wait...")

conn_request = CreateConnectionRequest()
conn_request.label = f'Present proof {datetime.date.today().strftime("%Y-%m-%d %H:%M:%S")}'
verifier_connection: Response[Connection] =  create_connection.sync(client=verifier_client,json_body=conn_request)

invitation = get_invitation_str(verifier_connection)

accept_conn_request = AcceptConnectionInvitationRequest(invitation)
holder_connection: Response[ConnectionInvitation] =  accept_connection_invitation.sync(client=holder_client,json_body=accept_conn_request)


verifier_connection: Response[Connection] = get_connection.sync(client=verifier_client,connection_id=verifier_connection.connection_id)
holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)

while (verifier_connection.state != 'ConnectionResponseSent' and holder_connection.state != 'ConnectionResponseReceived'):
    verifier_connection: Response[Connection] = get_connection.sync(client=verifier_client,connection_id=verifier_connection.connection_id)
    holder_connection: Response[Connection] = get_connection.sync(client=holder_client,connection_id=holder_connection.connection_id)
    print("Verifier State: {} / Holder State: {} \n".format(verifier_connection.state,holder_connection.state))
    time.sleep(1)
    
print("Connection established between verifier and Holder!")
print("\nVerifier connection:\n")
print_connection(verifier_connection)
print("\nHolder connection:\n")
print_connection(holder_connection)

Please wait...
Verifier State: InvitationGenerated / Holder State: ConnectionRequestPending 

Verifier State: InvitationGenerated / Holder State: ConnectionRequestPending 

Verifier State: ConnectionResponsePending / Holder State: ConnectionRequestPending 

Verifier State: ConnectionResponsePending / Holder State: ConnectionRequestSent 

Verifier State: ConnectionResponsePending / Holder State: ConnectionRequestSent 

Verifier State: ConnectionResponsePending / Holder State: ConnectionResponseReceived 

Connection established between verifier and Holder!

Verifier connection:

connection_id: 4eb2271c-c577-4d95-9aae-5efb3665169f
state:         ConnectionResponsePending
label:         Present proof 2023-01-31 00:00:00
my_did:        did:peer:2.Ez6LSm4kELijx9AJWAdF5cEu1s64MGnNCsAT3T74ChN3n3AGf.Vz6MkfqvXP89QraHWDySDjqnDJpzaJRiwtpGLQ2RkSn1uQNAS.SeyJ0IjoiZG0iLCJzIjoiaHR0cHM6Ly9rcjM0YS5hdGFsYXByaXNtLmlvL3ByaXNtLWFnZW50L2RpZGNvbW0iLCJyIjpbXSwiYSI6WyJkaWRjb21tL3YyIl19
their_did:     did:peer:2.

### Verifier - Create presentation request
Uses the connectionID of the connection with the Holder, proofs describe the credential requested 

In [5]:
data = {
    "connectionId": verifier_connection.connection_id,
    "proofs":[
        {
            "schemaId": "https://schema.org/Person",
            "trustIssuers": [
                "did:web:atalaprism.io/users/testUser"
            ]
        }
    ]
}

proof_request = RequestPresentationInput.from_dict(data)



### Verifier - Send proof request

In [6]:
verifier_proof_request: Response[RequestPresentationInput] = request_presentation.sync(client=verifier_client, json_body=proof_request)
print_proof_request(verifier_proof_request)

presentation_id: 9963ac55-365b-494e-9bcf-b7906aeedbbe


### Holder - Wait for proof request

In [7]:
holder_proof_request = find_proof_request_by_state(holder_client, "RequestReceived")

while(holder_proof_request == None):
    holder_proof_request = find_proof_request_by_state(holder_client, "RequestReceived")
    time.sleep(1)
    
print_proof_request(holder_proof_request)

presentation_id: 642a8314-b358-4bc0-9cb4-6d957acdee82
status:          RequestReceived
connection_id:   <prism_agent_open_api_specification_client.types.Unset object at 0x7f22367d8a90>


### Holder - Accept proof request

In [8]:
from typing import Any, Dict, Optional, Union, cast
action = RequestPresentationAction(action=RequestPresentationActionAction.REQUEST_ACCEPT, proof_id=["0c263de4-99c6-4134-bc77-323fa868d386"])

##Always returns none
update_presentation.sync(client=holder_client, json_body=action, id=holder_proof_request.presentation_id)

### Verifier - Wait for verification

In [9]:
verifier_proof_request: Response[PresentationStatus] = get_presentation.sync(client=verifier_client, id=verifier_proof_request.presentation_id)

while(verifier_proof_request.status != "PresentationVerified"):
    verifier_proof_request: Response[PresentationStatus] = get_presentation.sync(client=verifier_client, id=verifier_proof_request.presentation_id)
    print(f"status: {verifier_proof_request.status}")
    time.sleep(1)
    
print_proof_request(verifier_proof_request)

status: PresentationReceived
status: PresentationReceived
status: PresentationReceived
status: PresentationVerified
presentation_id: 9963ac55-365b-494e-9bcf-b7906aeedbbe
status:          PresentationVerified
connection_id:   <prism_agent_open_api_specification_client.types.Unset object at 0x7f22367d8a90>
